# PROJECT. Исследование поведения пользователей в приложении 
## В рамках этого исследования необходимо проверить:
- есть ли зависимость между выбранным уровнем сложности вопросов и вероятностью оплаты пакетов вопросов;
- различается ли временной промежуток между регистрацией и оплатой у групп пользователей с разным уровнем сложности.

Загружаем csv-файлы 7_4_Events.csv и purchase.csv, содержащие информацию  о событиях и оплатах, которые совершают пользователи.

In [173]:
import pandas as pd
events_df = pd.read_csv('data/game/7_4_Events.csv', sep = ',')
# выведем первые 10 строк датафрейма events_df, чтобы ознакомиться с данными
events_df.head(10)

,id,event_type,selected_level,start_time,tutorial_id,user_id
0,28903,registration,NaN,2016-05-11T23:40:55,NaN,12583
1,28904,registration,NaN,2016-05-11T23:49:58,NaN,12584
2,28905,registration,NaN,2016-05-12T00:53:07,NaN,12585
3,28906,tutorial_start,NaN,2016-05-12T01:32:20,17562.0,12585
4,28907,tutorial_finish,NaN,2016-05-12T01:34:53,17562.0,12585
5,28908,tutorial_start,NaN,2016-05-12T02:11:29,17563.0,12584
6,28909,tutorial_finish,NaN,2016-05-12T02:14:14,17563.0,12584
7,28910,level_choice,hard,2016-05-12T05:30:25,NaN,12584
8,28911,pack_choice,NaN,2016-05-12T05:38:24,NaN,12584
9,28912,tutorial_start,NaN,2016-05-12T07:19:01,17564.0,12583


In [174]:
# получим общую информацию о датафрейме, для того чтобы оценить, 
# какие данные содержатся в датафрейме events_df
events_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252334 entries, 0 to 252333
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              252334 non-null  int64  
 1   event_type      252334 non-null  object 
 2   selected_level  31086 non-null   object 
 3   start_time      252334 non-null  object 
 4   tutorial_id     125103 non-null  float64
 5   user_id         252334 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 11.6+ MB


Датафрейм events_df содержит 252334 строки и шесть столбцов. При этом все строки содержат значения в столбцах id, user_id, start_time, event_type, а столбцы selected_level и  tutorial_id содержат пропущенные значения.
Столбец start_time имеет тип данных object и для дальнейшего исследования нам необходимо будет перевести его в тип данных datetime.

In [175]:
purchase_df = pd.read_csv('data/game/purchase.csv', sep = ',')
# выведем первые 10 строк датафрейма purchase_df, чтобы ознакомиться с данными
purchase_df.head(10)


,id,user_id,event_datetime,amount
0,15674,12584,2016-05-12T10:34:16,100
1,15675,12985,2016-05-13T08:25:56,50
2,15676,12828,2016-05-13T16:33:46,50
3,15677,12598,2016-05-14T01:09:37,150
4,15678,13037,2016-05-14T01:24:46,100
5,15679,12989,2016-05-14T02:19:20,200
6,15680,12684,2016-05-14T03:10:14,100
7,15681,12656,2016-05-14T03:42:48,100
8,15682,12624,2016-05-14T06:00:45,50
9,15683,13136,2016-05-14T07:31:03,50


In [176]:
# получим общую информацию о датафрейме, для того чтобы оценить, 
# какие данные содержатся в датафрейме purchase_df
purchase_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5956 entries, 0 to 5955
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              5956 non-null   int64 
 1   user_id         5956 non-null   int64 
 2   event_datetime  5956 non-null   object
 3   amount          5956 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 186.2+ KB


Датафрейм не содержит пропусков, вся информация содержится в 4 столбцах id, user_id, event_datetime, amount. Для дальнейшего исследования нам необходимо будет перевести столбец event_datetime в тип datetime.

In [177]:
# Делаем выборку пользователей, зарегистрировавшихся 2018 году 
cond = (events_df.start_time>='2018-01-01') & (events_df.start_time<'2019-01-01') & (events_df.event_type=='registration')
# Список пользователей, зарегистрировавшихся в 2018
registered = events_df[cond]['user_id'].to_list() 
# Сохраняем полученный срез в переменную event
events = events_df[events_df.user_id.isin(registered)]
# Преобразуем тип данных столбца start_time в datetime
events.start_time = pd.to_datetime(events.start_time, format='%Y-%m-%dT%H:%M:%S') 
events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66959 entries, 51405 to 118364
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              66959 non-null  int64         
 1   event_type      66959 non-null  object        
 2   selected_level  8342 non-null   object        
 3   start_time      66959 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         66959 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 3.6+ MB


C:\Users\123\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [178]:
# Так как мы изучаем событие выбора уровня сложности вопросов на events,
# оставив в нём только такие строки, где event_type = level_choice
events[events['event_type'] == 'level_choice'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8342 entries, 51424 to 118363
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              8342 non-null   int64         
 1   event_type      8342 non-null   object        
 2   selected_level  8342 non-null   object        
 3   start_time      8342 non-null   datetime64[ns]
 4   tutorial_id     0 non-null      float64       
 5   user_id         8342 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 456.2+ KB


Как видим, этот срез содержит 8342 строки и в нем есть пропуски в tutorial_id. Это связано с тем, что для событий типа level_choice не предусмотрена запись параметра tutorial_id. Поэтому для дальнейшего исследования факт наличия пропусков в этом столбце нам не важен. 

In [179]:
# Посмотрим на первых 10 строк из датафрейма, в котором содержатся данные о пользователях, сделавших выбор уровня сложности вопросов
events[events['event_type'] == 'level_choice'].head(10)

,id,event_type,selected_level,start_time,tutorial_id,user_id
51424,80327,level_choice,medium,2018-01-01 20:37:22,NaN,27835
51428,80331,level_choice,hard,2018-01-01 22:37:50,NaN,27839
51441,80344,level_choice,medium,2018-01-02 05:18:42,NaN,27840
51443,80346,level_choice,hard,2018-01-02 06:19:18,NaN,27845
51450,80353,level_choice,easy,2018-01-02 08:46:03,NaN,27842
51457,80360,level_choice,easy,2018-01-02 11:53:11,NaN,27849
51462,80365,level_choice,medium,2018-01-02 14:09:58,NaN,27843
51464,80367,level_choice,medium,2018-01-02 15:10:27,NaN,27846
51473,80376,level_choice,medium,2018-01-02 18:09:02,NaN,27847
51475,80378,level_choice,medium,2018-01-02 18:39:44,NaN,27852


In [180]:
# Оценим, какие уникальные события есть в столбце selected_level
events['selected_level'].unique()

array([nan, 'medium', 'hard', 'easy'], dtype=object)

In [181]:
# Оценим, какое количество пользователей совершали события
events['user_id'].nunique()


19926

In [182]:
# Делаем выборку оплаты событий, совершенных пользователями, зарегистрировавшихся 2018 году
purchase = purchase_df[purchase_df['user_id'].isin(registered)]
# Изменим тип столбца  event_datetime на datetime
purchase.event_datetime = pd.to_datetime(purchase.event_datetime, format='%Y-%m-%dT%H:%M:%S')

C:\Users\123\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [184]:
# Определим количество пользователей, перешедших на выбор уровня сложности вопросов
events[events['event_type'] == 'level_choice']['user_id'].nunique()

8342

Посчитаем, какой процент от общего количества зарегистрировавшихся пользователей перешли к уровню выбора уровня сложности вопросов

In [185]:
registered_users_count = events[events['event_type'] == 'registration'][
    'user_id'
].nunique()
level_choice_users_count = events[events['event_type'] == 'level_choice'][
    'user_id'
].nunique()
percent_tutorial_start_users = level_choice_users_count / registered_users_count
print("Процент пользователей, выбравших уровень сложности тренировок (от общего числа зарегистрировавшихся): {:.2%}".format(
percent_tutorial_start_users
    )
)

Процент пользователей, выбравших уровень сложности тренировок (от общего числа зарегистрировавшихся): 41.86%


Посчитаем процент пользователей, купивших платные вопросы, от числа пользователей, которые выбрали уровень сложности

In [186]:
# Количество пользователей, которые оплатили вопросы
paying_users_count = purchase["user_id"].nunique()
percent_of_paying_users = paying_users_count / level_choice_users_count
print(
    "Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень сложности): {:.2%}".format(
        percent_of_paying_users
    )
)

Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень сложности): 19.18%


Добавим в датафрейм purchase столбец event_type, который будет содержать одно значение purchase. Это нужно, чтобы в объединённом датафрейме однозначно выделить события оплаты.

In [187]:
purchase['event_type'] = 'purchase'

C:\Users\123\AppData\Local\Temp/ipykernel_17232/1009542599.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchase['event_type'] = 'purchase'


Также у нас есть одинаковые столбцы id в двух датафреймах, но смысл их несколько отличается, так как столбец id в events указывает на идентификатор события, а столбец id в purchase указывает на идентификатор оплаты. Поэтому применим функцию rename(), чтобы переименовать столбцы в датафреймах. Также переименуем столбец event_datetime в purchase_datetime

In [188]:
events = events.rename(columns={"id": "event_id"})
purchase = purchase.rename(columns={"id": "purchase_id", "event_datetime": "purchase_datetime"})


Следующим шагом объединим датафреймы events и purchase с помощью функции pd.concat() и запишем объединённый датафрейм в переменную total_events_df.

In [189]:
total_events_df = pd.concat([events,purchase],sort=False)

In [190]:
# Посмотрим, что получилось в итоге объединения
total_events_df.head(10)

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,purchase_datetime,amount
51405,80308.0,registration,NaN,2018-01-01 03:48:40,NaN,27832,NaN,NaT,NaN
51406,80309.0,registration,NaN,2018-01-01 04:07:25,NaN,27833,NaN,NaT,NaN
51407,80310.0,registration,NaN,2018-01-01 08:35:10,NaN,27834,NaN,NaT,NaN
51408,80311.0,registration,NaN,2018-01-01 11:54:47,NaN,27835,NaN,NaT,NaN
51409,80312.0,registration,NaN,2018-01-01 13:28:07,NaN,27836,NaN,NaT,NaN
51410,80313.0,registration,NaN,2018-01-01 14:08:40,NaN,27837,NaN,NaT,NaN
51411,80314.0,registration,NaN,2018-01-01 14:42:58,NaN,27838,NaN,NaT,NaN
51412,80315.0,tutorial_start,NaN,2018-01-01 14:54:40,31505.0,27836,NaN,NaT,NaN
51413,80316.0,tutorial_start,NaN,2018-01-01 15:00:51,31506.0,27835,NaN,NaT,NaN
51414,80317.0,tutorial_finish,NaN,2018-01-01 15:06:15,31506.0,27835,NaN,NaT,NaN


In [191]:
# Разделим пользователей на группы в зависимости от выбранного уровня сложности (easy, medium, hard)
users_easy_level = total_events_df[total_events_df['selected_level'] == 'easy']["user_id"].unique()
users_medium_level = total_events_df[total_events_df['selected_level'] == 'medium']["user_id"].unique()
users_hard_level = total_events_df[total_events_df['selected_level'] == 'hard']["user_id"].unique()

In [192]:
# Посчитаем общее количество пользователей, совершивших покупки
paying_users_count = purchase["user_id"].nunique()
print(paying_users_count)

1600


In [193]:
# Определим количество пользователей, купивших вопросы уровня 'easy'
purchase_easy_level = purchase[purchase["user_id"].isin(users_easy_level)]
purchase_easy_level['user_id'].nunique()

189

In [194]:
# Определим количество пользователей, купивших вопросы уровня 'medium'
purchase_medium_level = purchase[purchase["user_id"].isin(users_medium_level)]
purchase_medium_level['user_id'].nunique()

969

In [195]:
# Определим количество пользователей, купивших вопросы уровня 'hard'
purchase_hard_level = purchase[purchase["user_id"].isin(users_hard_level)]
purchase_hard_level['user_id'].nunique()

442

In [196]:
# Сверим общее количество пользователей, совершивших покупки с общим количеством пользователей,
# которых мы разбили на группы по уровню сложности вопросов
total = purchase_hard_level['user_id'].nunique() + purchase_medium_level['user_id'].nunique() + purchase_easy_level['user_id'].nunique()
print(total)
print(paying_users_count == total)

1600
True


Далее определим, какой процент от общего количества пользователей, оплативших покупки, составляет каждая группа 

In [197]:
easy_level_paying = purchase_easy_level['user_id'].nunique()/paying_users_count
print(
    "Процент пользователей, которые выбрали легкий уровень и оплатили тренировки (от числа всех пользователей, совершивших покупки): {:.2%}".format(
        easy_level_paying
    )
)

Процент пользователей, которые выбрали легкий уровень и оплатили тренировки (от числа всех пользователей, совершивших покупки): 11.81%


In [198]:

medium_level_paying = purchase_medium_level['user_id'].nunique()/paying_users_count
print(
    "Процент пользователей, которые выбрали средний уровень и оплатили тренировки (от числа всех пользователей, совершивших покупки): {:.2%}".format(
        medium_level_paying
    )
)

Процент пользователей, которые выбрали средний уровень и оплатили тренировки (от числа всех пользователей, совершивших покупки): 60.56%


In [199]:
hard_level_paying = purchase_hard_level['user_id'].nunique()/paying_users_count
print(
    "Процент пользователей, которые выбрали тяжелый уровень и оплатили тренировки (от числа всех пользователей, совершивших покупки): {:.2%}".format(
        hard_level_paying
    )
)

Процент пользователей, которые выбрали тяжелый уровень и оплатили тренировки (от числа всех пользователей, совершивших покупки): 27.62%


Как видим из результатов расчетов, наибольший процент от общего числа покупателей, составляют пользователи, которые выбрали средний уровень вопросов (60.56 %), а наименьший - пользователи, выбравшие легкий уровень (11.81 %). Тяжелый уровень вопросов выбрали 27.62 % покупателей

Далее найдем процент пользователей, которые выбрали определенный уровень и оплатили тренировки, от числа всех пользователей, выбравших данный уровень

In [200]:

hard_level_paying = purchase_hard_level['user_id'].nunique()/len(users_hard_level)
print(
    "Процент пользователей, которые выбрали уровень hard и оплатили тренировки (от числа всех пользователей, выбравших уровень hard): {:.2%}".format(
        hard_level_paying
    )
)

Процент пользователей, которые выбрали уровень hard и оплатили тренировки (от числа всех пользователей, выбравших уровень hard): 35.39%


In [201]:
medium_level_paying = purchase_medium_level['user_id'].nunique()/len(users_medium_level)
print(
    "Процент пользователей, которые выбрали уровень medium и оплатили тренировки (от числа всех пользователей, выбравших уровень medium): {:.2%}".format(
        medium_level_paying
    )
)

Процент пользователей, которые выбрали уровень medium и оплатили тренировки (от числа всех пользователей, выбравших уровень medium): 20.86%


In [202]:
easy_level_paying = purchase_easy_level['user_id'].nunique()/len(users_easy_level)
print(
    "Процент пользователей, которые выбрали уровень easy и оплатили тренировки (от числа всех пользователей, выбравших уровень easy): {:.2%}".format(
        easy_level_paying
    )
)

Процент пользователей, которые выбрали уровень easy и оплатили тренировки (от числа всех пользователей, выбравших уровень easy): 7.72%


Как видим из полученных результатов, вероятность покупки платных вопросов выше у пользователей, выбравших сложные вопросы и вопросы среднего уровня ( 35.39% и 20.86% соответственно). И только лишь  7.72 % пользователей, выбравших легкий уровень вопросов, переходят к выбору платных пакетов. Необходимо изучить, почему у пользователей, выбравших легкий уровень, такая низкая заинтересованность приложением. Возможно, это связано с тем, что таким пользователям достаточно вопросов, доступных в бесплатном формате, либо вопросы показались слишком сложными.

Далее посчитаем, сколько времени проходит между регистрацией и выбором уровня сложности. 
Первым делом выделим отдельный датафрейм registration_df, который будет содержать только события с event_type = registration. Этот датафрейм будет вспомогательным для определения времени между регистрацией и выбором уровня сложности

In [203]:
registration_df = total_events_df[total_events_df['event_type'] == 'registration']

Проверим, что в нашем датафрейме registration_df для каждого пользователя содержится только по одному событию, то есть по одной регистрации на пользователя. Это важно, чтобы однозначно для одного пользователя присоединить только одного событие, по которому мы будем считать время.

In [204]:
registration_df['user_id'].value_counts().mean()

1.0

Оставим в датафрейме registration_df только те данные, которые нужны для наших вычислений — столбец user_id с идентификатором пользователя и столбец start_time со временем регистрации. Также переименуем столбец start_time в столбец registration_time для понятности.

In [205]:
registration_df = registration_df[["user_id", "start_time"]].rename(
    columns={"start_time": "registration_time"}
)

In [206]:
# Выделим отдельный датафрейм choice_level, который будет содержать только события с event_type = level_choice
choice_level_df = total_events_df[total_events_df['event_type'] == 'level_choice']

In [207]:
# Проверим, что в нашем датафрейме choice_level_df для каждого пользователя содержится только по одному событию
choice_level_df['user_id'].value_counts().mean()

1.0

In [208]:
# Переименуем столбец start_time на choice_level_start_time
choice_level_df = choice_level_df[["user_id", "start_time"]].rename(columns={"start_time": "choice_level_start_time"})
# Проверим результат переименования
print(choice_level_df.head(10))

       user_id choice_level_start_time
51424    27835     2018-01-01 20:37:22
51428    27839     2018-01-01 22:37:50
51441    27840     2018-01-02 05:18:42
51443    27845     2018-01-02 06:19:18
51450    27842     2018-01-02 08:46:03
51457    27849     2018-01-02 11:53:11
51462    27843     2018-01-02 14:09:58
51464    27846     2018-01-02 15:10:27
51473    27847     2018-01-02 18:09:02
51475    27852     2018-01-02 18:39:44


In [209]:
# Создадим отдельные датафреймы пользователей в зависимости от уровня вопросов
easy_level_df = choice_level_df[choice_level_df["user_id"].isin(users_easy_level)]
medium_level_df = choice_level_df[choice_level_df["user_id"].isin(users_medium_level)]
hard_level_df = choice_level_df[choice_level_df["user_id"].isin(users_hard_level)]


In [210]:
# Объединим registration_df и easy_level_df, merged_medium_df, merged_hard_df. Это позволит для каждого пользователя в рамках одного датафрейма указать и время регистрации,
# и время выбора уровня сложности вопросов в приложении.
merged_easy_df = registration_df.merge(easy_level_df, on="user_id", how="inner")
merged_medium_df = registration_df.merge(medium_level_df, on="user_id", how="inner")
merged_hard_df = registration_df.merge(hard_level_df, on="user_id", how="inner")
# Проверим, что получилось в результате объединения
merged_easy_df.head()

,user_id,registration_time,choice_level_start_time
0,27842,2018-01-01 23:18:46,2018-01-02 08:46:03
1,27849,2018-01-02 07:03:55,2018-01-02 11:53:11
2,27853,2018-01-02 15:47:14,2018-01-03 00:39:54
3,27859,2018-01-02 21:30:27,2018-01-03 01:11:45
4,27863,2018-01-03 03:53:48,2018-01-03 09:31:00


In [211]:
# Определим разницу во времени между регистрацией и выбором уровня 
merged_easy_df["timedelta"] = (merged_easy_df["choice_level_start_time"] - merged_easy_df["registration_time"])
merged_medium_df["timedelta"] = (merged_medium_df["choice_level_start_time"] - merged_medium_df["registration_time"])
merged_hard_df["timedelta"] = (merged_hard_df["choice_level_start_time"] - merged_hard_df["registration_time"])
# Проверяем, что получилось в результате объединения
merged_easy_df.head()

,user_id,registration_time,choice_level_start_time,timedelta
0,27842,2018-01-01 23:18:46,2018-01-02 08:46:03,0 days 09:27:17
1,27849,2018-01-02 07:03:55,2018-01-02 11:53:11,0 days 04:49:16
2,27853,2018-01-02 15:47:14,2018-01-03 00:39:54,0 days 08:52:40
3,27859,2018-01-02 21:30:27,2018-01-03 01:11:45,0 days 03:41:18
4,27863,2018-01-03 03:53:48,2018-01-03 09:31:00,0 days 05:37:12


In [212]:
# Среднее время, которое проходит между регистрацией пользователя и выбором уровня сложности
merged_easy_df["timedelta"].mean()
merged_medium_df["timedelta"].mean()
merged_hard_df["timedelta"].mean()
print('Среднее время между регистрацией и выбором уровня easy: ', merged_easy_df["timedelta"].mean())
print('Среднее время между регистрацией и выбором уровня medium: ', merged_medium_df["timedelta"].mean())
print('Среднее время между регистрацией и выбором уровня hard: ', merged_hard_df["timedelta"].mean())


Среднее время между регистрацией и выбором уровня easy:  0 days 07:10:06.324754901
Среднее время между регистрацией и выбором уровня medium:  0 days 07:08:13.360387513
Среднее время между регистрацией и выбором уровня hard:  0 days 07:18:32.228182546


Как видим, значимой разницы во времени между регистрацией и выбором определенного уровня сложности вопросов нет. В среднем пользователи выбирают уровень сложности вопросов через 7 часов с момента регистрации.

Далее изучим, есть ли разница во времени между регистрацией и покупкой вопросов разными группами пользователей

In [213]:
# Объединяем между собой датафреймы: registration_df и purchase_easy_level, registration_df и purchase_medium_level, 
# registration_df и purchase_hard_level
merged_easy_purch_df = registration_df.merge(purchase_easy_level, on="user_id", how="inner")
merged_medium_purch_df = registration_df.merge(purchase_medium_level, on="user_id", how="inner")
merged_hard_purch_df = registration_df.merge(purchase_hard_level, on="user_id", how="inner")
# Проверяем, что получилось в результате объединения
merged_easy_purch_df.head()

,user_id,registration_time,purchase_id,purchase_datetime,amount,event_type
0,27884,2018-01-04 11:50:43,16854,2018-01-08 19:37:34,150,purchase
1,28090,2018-01-09 19:31:24,16873,2018-01-15 23:42:55,100,purchase
2,28182,2018-01-11 10:12:20,16865,2018-01-12 02:46:01,200,purchase
3,28207,2018-01-11 16:27:37,16867,2018-01-12 21:00:24,150,purchase
4,28247,2018-01-12 10:01:12,16884,2018-01-18 18:32:05,150,purchase


In [214]:
# Находим разницу во времени
merged_easy_purch_df["timedelta"] = (merged_easy_purch_df["purchase_datetime"] - merged_easy_purch_df["registration_time"])
merged_medium_purch_df["timedelta"] = (merged_medium_purch_df["purchase_datetime"] - merged_medium_purch_df["registration_time"])
merged_hard_purch_df["timedelta"] = (merged_hard_purch_df["purchase_datetime"] - merged_hard_purch_df["registration_time"])
# Проверяем, что получилось в результате объединения
merged_easy_purch_df.head()

,user_id,registration_time,purchase_id,purchase_datetime,amount,event_type,timedelta
0,27884,2018-01-04 11:50:43,16854,2018-01-08 19:37:34,150,purchase,4 days 07:46:51
1,28090,2018-01-09 19:31:24,16873,2018-01-15 23:42:55,100,purchase,6 days 04:11:31
2,28182,2018-01-11 10:12:20,16865,2018-01-12 02:46:01,200,purchase,0 days 16:33:41
3,28207,2018-01-11 16:27:37,16867,2018-01-12 21:00:24,150,purchase,1 days 04:32:47
4,28247,2018-01-12 10:01:12,16884,2018-01-18 18:32:05,150,purchase,6 days 08:30:53


In [215]:
# Находим усредненное значение
merged_easy_purch_df["timedelta"].mean()
merged_medium_purch_df["timedelta"].mean()
merged_hard_purch_df["timedelta"].mean()
print('Среднее время между регистрацией и покупкой вопросов, пользователями, выбравшими уровнь easy: ', merged_easy_purch_df["timedelta"].mean())
print('Среднее время между регистрацией и покупкой вопросов, пользователями, выбравшими уровнь medium: ', merged_medium_purch_df["timedelta"].mean())
print('Среднее время между регистрацией и покупкой вопросов, пользователями, выбравшими уровнь hard: ', merged_hard_purch_df["timedelta"].mean())

Среднее время между регистрацией и покупкой вопросов, пользователями, выбравшими уровнь easy:  3 days 22:10:23.211640211
Среднее время между регистрацией и покупкой вопросов, пользователями, выбравшими уровнь medium:  4 days 06:12:06.576883384
Среднее время между регистрацией и покупкой вопросов, пользователями, выбравшими уровнь hard:  3 days 14:55:19.257918552


Как видим из расчетов, пользователи, выбравшие легкий уровень вопросов и тяжелый уровень, покупают платные пакеты в среднем через 3 дня с момента регистрации. А вот пользователям, выбравшим средний уровень вопросов, необходимо на день больше времени.

Найдем разницу во времени между выбором уровня сложности вопросов и покупкой платных вопросов.

In [216]:
#Объединим датафрейм категории пользователей с датафреймом покупок данной категории
merged_easy_purch_2df = easy_level_df.merge(purchase_easy_level, on="user_id", how="inner")
merged_medium_purch_2df = medium_level_df.merge(purchase_medium_level, on="user_id", how="inner")
merged_hard_purch_2df = hard_level_df.merge(purchase_hard_level, on="user_id", how="inner")
# Посмотрим на результат объединения
merged_easy_purch_2df.head()

,user_id,choice_level_start_time,purchase_id,purchase_datetime,amount,event_type
0,27884,2018-01-04 16:18:39,16854,2018-01-08 19:37:34,150,purchase
1,28090,2018-01-09 21:34:23,16873,2018-01-15 23:42:55,100,purchase
2,28182,2018-01-11 18:44:45,16865,2018-01-12 02:46:01,200,purchase
3,28207,2018-01-11 21:10:51,16867,2018-01-12 21:00:24,150,purchase
4,28254,2018-01-12 16:48:24,16892,2018-01-19 22:08:40,50,purchase


In [217]:
# Находим разницу во времени
merged_easy_purch_2df["timedelta"] = (merged_easy_purch_2df["purchase_datetime"] - merged_easy_purch_2df["choice_level_start_time"])
merged_medium_purch_2df["timedelta"] = (merged_medium_purch_2df["purchase_datetime"] - merged_medium_purch_2df["choice_level_start_time"])
merged_hard_purch_2df["timedelta"] = (merged_hard_purch_2df["purchase_datetime"] - merged_hard_purch_2df["choice_level_start_time"]
)
merged_easy_purch_2df.head()

,user_id,choice_level_start_time,purchase_id,purchase_datetime,amount,event_type,timedelta
0,27884,2018-01-04 16:18:39,16854,2018-01-08 19:37:34,150,purchase,4 days 03:18:55
1,28090,2018-01-09 21:34:23,16873,2018-01-15 23:42:55,100,purchase,6 days 02:08:32
2,28182,2018-01-11 18:44:45,16865,2018-01-12 02:46:01,200,purchase,0 days 08:01:16
3,28207,2018-01-11 21:10:51,16867,2018-01-12 21:00:24,150,purchase,0 days 23:49:33
4,28254,2018-01-12 16:48:24,16892,2018-01-19 22:08:40,50,purchase,7 days 05:20:16


In [218]:
# Находим среднее значение
merged_easy_purch_2df["timedelta"].mean()
merged_medium_purch_2df["timedelta"].mean()
merged_hard_purch_2df["timedelta"].mean()
print('Среднее время между выбором уровня сложности easy и покупкой вопросов: ',merged_easy_purch_2df["timedelta"].mean())
print('Среднее время между выбором уровня сложности medium и покупкой вопросов: ',merged_medium_purch_2df["timedelta"].mean())
print('Среднее время между выбором уровня сложности hard и покупкой вопросов: ',merged_hard_purch_2df["timedelta"].mean())

Среднее время между выбором уровня сложности easy и покупкой вопросов:  3 days 14:58:52.941798941
Среднее время между выбором уровня сложности medium и покупкой вопросов:  3 days 23:14:13.165118679
Среднее время между выбором уровня сложности hard и покупкой вопросов:  3 days 07:20:41.420814479


Разницы во времени между выбором уровня сложности и покупкой вопросов нет. Покупка совершается в среднем через три дня после выбора уровня сложности